### Creating Students Profiles

Imports

In [1]:
from faker import Faker
import pandas as pd
import random

fake = Faker()


PRE-Provided Data

In [11]:

departments = ["Computer Science" , "Electrical" , "Mechanical" , "Civil"]
students = []
years = [1,2,3,4]

Generating Data using faker

In [12]:

for i in range(101): #no of students 100
   students.append({
                   "Student_id":f"U{i:03d}",
                   "Name": fake.name(),
                   "Departments":random.choice(departments),
                   "Current_Year":random.choice(years)
   }
   )

Exporting the .csv

### Creating and assigning mentors

In [14]:
academic_mentors = [f"A{j:03d}" for j in range(10)]
industry_mentors = [f"I{j:03d}" for j in range(10)]

for student in students:                           #assigning the mentors
    student["academic_mentor"] = random.choice(academic_mentors)
    student["industry_mentor"] = random.choice(industry_mentors)
    
    

In [ ]:
students_df= pd.DataFrame(students) #Converting the raw data in the form of Dataframe/Tables

###print(students_df.head(5))


students_df.to_csv("students_with_mentors.csv", index=False)
print("CSV file saved: student_with_mentors.csv")

CSV file saved: student_with_mentors.csv
